In [ ]:
def ode_system(t, y, params):
    H = y[0]
    P = y[1]
    B = y[2]

    muH = params[0]
    muB = params[1]
    k = params[2]
    phi = params[3]
    beta = params[4]

    dHdt = (muH * H * (1 - (H / k) )) - phi * H * P
    dPdt = beta * phi * H * P
    dBdt = muB * B
    return [dHdt, dPdt, dBdt]

import numpy as np
from scipy.integrate import solve_ivp

def solve_ode_instance(times, theta, y0, method="LSODA", rtol=1e-6, atol=1e-8):
    times = np.asarray(times, dtype=float)
    theta = np.asarray(theta, dtype=float)
    y0 = np.asarray(y0, dtype=float)
    eval = times
    #eval = times[1:]

    sol = solve_ivp(
        fun=lambda t, y: ode_system(t, y, theta),
        t_span=(times[0], times[-1]),
        y0=y0,
        t_eval=eval,
        method=method,
        rtol=rtol,
        atol=atol,
    )

    if not sol.success:
        raise RuntimeError(f"ODE solver failed: {sol.message}")

    # Match your Op output: (time, 3)
    return sol.y.T, sol


In [2]:
muH, muB, k, phi, beta = 0.86, 0.68, 5.5e6, 7e-9, 24.1
theta = [muH, muB, k, phi, beta]

H0, P0, B0 = 6.8e6, 9.5e6, 4.4e5
y0 = [H0, P0, B0]

time = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

y_hat, sol = solve_ode_instance(time, theta, y0)
# y_hat.shape == (len(time), 3)

H_hat = y_hat[:, 0]
P_hat = y_hat[:, 1]
B_hat = y_hat[:, 2]

In [3]:
import numpy as np
import pandas as pd

path = "avg/"
sub_path = "d7/0-dmsp.xlsx"

host_path = "data/algae/" + path + sub_path
bacteria_path = "data/bacteria/" + path + sub_path

df_host = pd.read_excel(host_path, header=None)
df_bacteria = pd.read_excel(bacteria_path, header=None)
df_bacteria = df_bacteria.replace(0, np.nan)

df_host_time = df_host.iloc[:, 0]
host = df_host.iloc[:, 1].tolist()

df_bacteria_time = df_bacteria.iloc[:, 0]
bacteria = df_bacteria.iloc[:, 1].tolist()

time = df_host_time.tolist()

In [12]:
print(np.asarray(host))
print(np.asarray(H_hat))

[ 478460.57441567  690421.07969915 1203158.42785524 2009834.66010355
 3408558.07299458 5229445.13390678 5554773.37414749 4793534.14383127
 4089747.12725955 2305469.24801042 1850179.61529333   38379.34273659]
[6.80000000e+06 5.50883922e+06 4.44011705e+06 3.12792690e+06
 1.79631864e+06 8.56923196e+05 3.64307601e+05 1.46464410e+05
 5.74989960e+04 2.23580381e+04 8.66113813e+03 3.35029152e+03]


In [13]:
print(np.asarray(bacteria))
print(np.asarray(B_hat))

[1.00000000e+03 2.24930137e+05 3.52125981e+06 4.59083907e+06
 5.91554221e+06 8.31265972e+06 2.22042142e+07 4.84931389e+07
 7.95083735e+07 1.19082533e+08 1.12967209e+08 1.60136253e+08]
[4.40000000e+05 8.68506579e+05 1.71432606e+06 3.38387082e+06
 6.67934760e+06 1.31842159e+07 2.60240345e+07 5.13682748e+07
 1.01394743e+08 2.00140917e+08 3.95053771e+08 7.79787863e+08]


In [14]:
print(np.asarray(P_hat))

[9.50000000e+06 2.65454379e+07 6.15766186e+07 1.16971700e+08
 1.76643393e+08 2.19419471e+08 2.41965539e+08 2.51953758e+08
 2.56035602e+08 2.57648324e+08 2.58276977e+08 2.58520740e+08]
